In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LEVEL'] = '2'

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# 파일 경로/파일 개수 설정

In [3]:
tumor_yes_path = './yes/'
tumor_no_path = './no/'

tumor_yes_file_list = os.listdir(tumor_yes_path)
tumor_no_file_list = os.listdir(tumor_no_path)

tumor_yes_len = len(tumor_yes_file_list)
tumor_no_len = len(tumor_no_file_list)

total_len = tumor_yes_len + tumor_no_len

# 이미지 전처리

In [4]:
num = 0
all_img = np.float32(np.zeros((total_len, 224, 224, 3)))
all_label = np.float64(np.zeros((total_len, 2)))

for img_name in tumor_yes_file_list:
    img_path = tumor_yes_path + img_name
    img = load_img(img_path, target_size=(224, 224, 3))

    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :] = x

    all_label[num][1] = 1
    num += 1

for img_name in tumor_no_file_list:
    img_path = tumor_no_path + img_name
    img = load_img(img_path, target_size=(224, 224))

    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :] = x

    all_label[num][0] = 1
    num += 1

train_input, test_input, train_target, test_target = train_test_split(
    all_img, all_label, test_size=0.2
)

In [5]:
IMG_SHAPE = (224, 224, 3)
num_classes = 2

# 네트워크 구성

In [15]:
base_model = ResNet50(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))

model_lr = 0.0007

In [16]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_4 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               12845184  
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 258       
Total params: 36,433,666
Trainable params: 12,845,698
Non-trainable params: 23,587,968
_________________________________________________________________


In [17]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=model_lr),
    loss = 'binary_crossentropy',
    metrics=['accuracy']
)

model.fit(train_input, train_target, epochs=10, batch_size=16, validation_data=(test_input, test_target), shuffle=True)
model.save('modelv3.h5')

print('Success Save Model!')

Epoch 1/10
13/13 [==============================] - 8s 620ms/step - loss: 0.6489 - accuracy: 0.7673 - val_loss: 1.9116 - val_accuracy: 0.7843
Epoch 2/10
13/13 [==============================] - 7s 568ms/step - loss: 0.1411 - accuracy: 0.9455 - val_loss: 1.2847 - val_accuracy: 0.8431
Epoch 3/10
13/13 [==============================] - 7s 560ms/step - loss: 0.1231 - accuracy: 0.9455 - val_loss: 1.0401 - val_accuracy: 0.8235
Epoch 4/10
13/13 [==============================] - 7s 559ms/step - loss: 0.0479 - accuracy: 0.9901 - val_loss: 0.8587 - val_accuracy: 0.8627
Epoch 5/10
13/13 [==============================] - 7s 559ms/step - loss: 0.0297 - accuracy: 1.0000 - val_loss: 0.7789 - val_accuracy: 0.8627
Epoch 6/10
13/13 [==============================] - 7s 559ms/step - loss: 0.0506 - accuracy: 0.9851 - val_loss: 0.7693 - val_accuracy: 0.8627
Epoch 7/10
13/13 [==============================] - 7s 560ms/step - loss: 0.0238 - accuracy: 0.9950 - val_loss: 0.7111 - val_accuracy: 0.8431
Epoch 